In [ ]:
using OrdinaryDiffEq
using SciMLSensitivity
using Enzyme
using LinearAlgebra

# 土壤水力参数 (这是我们要微分的对象 -> Active)
struct VGParam{T}
    θ_s::T
    θ_r::T
    Ks::T  # [cm/h]
    α::T   # [1/cm]
    n::T   # [-]
end

# 网格配置 (常量 -> Inactive)
struct SoilConfig{T}
    N::Int
    Δz::Vector{T}      # [cm]
    Δz_half::Vector{T} # [cm]
    depths::Vector{T}
end

# 运行时缓存 (预分配内存 -> Cache)
struct SoilCache{T}
    K::Vector{T}
    K_half::Vector{T}
    ψ::Vector{T}
    Q::Vector{T}
    sink::Vector{T}
end

# 工厂函数：初始化 Soil
function make_soil(n_layers::Int, dz_val::T) where T
    depths = collect(range(dz_val, length=n_layers, step=dz_val))
    Δz = fill(dz_val, n_layers)
    Δz_half = copy(Δz) 
    
    cfg = SoilConfig(n_layers, Δz, Δz_half, depths)
    
    cache = SoilCache(
        zeros(T, n_layers),      # K
        zeros(T, n_layers),      # K_half
        zeros(T, n_layers),      # ψ
        zeros(T, n_layers + 1),  # Q (N+1 interfaces)
        zeros(T, n_layers)       # sink
    )
    return cfg, cache
end

# Van Genuchten 模型计算
@inline function vg_hydraulic(θ, p::VGParam{T}) where T
    # 物理约束：防止 log(0) 或 负数
    S_e = (θ - p.θ_r) / (p.θ_s - p.θ_r)
    S_e = clamp(S_e, T(1e-4), T(1.0 - 1e-4)) 
    
    m = 1 - 1/p.n
    # 计算 ψ (吸力 head, cm)
    head = - (1/p.α) * (S_e^(-1/m) - 1)^(1/p.n)
    
    # 计算 K (导水率, cm/h)
    denom = (1 - S_e^(1/m))^m
    k_val = p.Ks * sqrt(S_e) * (1 - denom)^2 
    return k_val, head
end

# 计算通量 (In-Place 修改 Cache)
function cal_Q!(cache::SoilCache, config::SoilConfig, ps::Vector{VGParam{T}}, θ, t) where T
    N = config.N
    # 1. 更新 K 和 ψ
    @inbounds for i in 1:N
        K, ψ = vg_hydraulic(θ[i], ps[i])
        cache.K[i] = K
        cache.ψ[i] = ψ
    end

    # 计算交界面 K (算术平均)
    @inbounds for i in 1:N-1
        cache.K_half[i] = (cache.K[i] + cache.K[i+1]) / 2 
    end

    # 2. 上边界 (假设恒定入渗)
    Q_top = -1.0 # [cm/h] 向下
    cache.Q[1] = Q_top

    # 3. 内部通量 (Darcy Law)
    # Q = -K * (dψ/dz + 1)
    @inbounds for i in 1:N-1
        # 简单的差分格式
        dψ_dz = (cache.ψ[i+1] - cache.ψ[i]) / config.Δz_half[i] 
        # 加上重力项 (+1.0)
        q_val = -cache.K_half[i] * ( dψ_dz + 1.0 )
        cache.Q[i+1] = q_val
    end

    # 4. 下边界 (自由排水)
    cache.Q[N+1] = -cache.K[N] 
    
    return nothing
end

# 实现调用方法，使其像函数一样工作: f(du, u, p, t)
function (sys::RichardsSystem)(dθ, θ, ps, t, config, cache)
    # 从 self 中获取常量和缓存
    config = sys.config
    cache = sys.cache
    
    # 1. 计算中间变量 (Flux)
    cal_Q!(cache, config, ps, θ, t)
    
    # 2. 计算 dθ/dt = -dQ/dz
    dθ[1] = -(cache.Q[2] - cache.Q[1]) / config.Δz[1]
    N = config.N
    @inbounds for i in 2:N
        dθ[i] = -(cache.Q[i+1] - cache.Q[i]) / config.Δz[i]
    end
    return nothing
end

In [ ]:
n_layers = 10
dz = 5.0
config, cache = make_soil(n_layers, dz)
sys = RichardsSystem(config, cache)

x = [8.0, 1.8]
val_Ks = x[1]
val_n  = x[2]
ps = [VGParam(0.45, 0.05, val_Ks, 0.01, val_n) for _ in 1:n_layers]

10-element Vector{VGParam{Float64}}:
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)

In [ ]:
using ComponentArrays
ComponentArray(ps)

10-element Vector{VGParam{Float64}}:
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)
 VGParam{Float64}(0.45, 0.05, 8.0, 0.01, 1.8)

In [ ]:


θ0 = fill(0.3, n_layers)
tspan = (0.0, 2.0) # 模拟2小时

# 输入 x 是我们想优化的参数 (例如 Ks 和 n)
function loss_function(x)
    val_Ks = x[1]
    val_n  = x[2]
    
    # 构造完整的参数对象 (Active)
    # 所有的层使用相同的参数，方便测试
    ps = [VGParam(0.45, 0.05, val_Ks, 0.01, val_n) for _ in 1:n_layers]
    
    # 创建 ODEProblem
    # 注意：这里的 p 只有 ps，没有 config/cache
    prob = ODEProblem(sys, θ0, tspan, ps)
    
    # 设置敏感度算法
    # InterpolatingAdjoint: 更稳定
    # autojacvec=EnzymeVJP(): 全栈使用 Enzyme
    sensalg = InterpolatingAdjoint(autojacvec=EnzymeVJP())
    
    # 求解 (降低一点精度以加快测试)
    sol = solve(prob, Tsit5(), saveat=0.5, sensealg=sensalg, abstol=1e-4, reltol=1e-4)
    
    # Loss: 简单的求和，只要能产生梯度即可
    return sum(sol)
end
# 初始猜测值 [Ks, n]
p_in = [8.0, 1.8]

# 梯度容器
dp = zeros(length(p_in))
println("System initialized. Starting Enzyme AD...")
println("Initial parameters: Ks=$(p_in[1]), n=$(p_in[2])")

# 调用 Enzyme
Enzyme.autodiff(
    set_runtime_activity(Reverse), 
    # loss_function, 
    Enzyme.Const(loss_function),
    Enzyme.Active, 
    Enzyme.Duplicated(p_in, dp)
)
println("\n=== Result ===")
println("dLoss / dKs = ", dp[1])
println("dLoss / dn  = ", dp[2])